### 0.1. Importação de Bibliotecas

Primeiramente, importamos todas as bibliotecas necessárias para a execução do código. Nesse caso, usamos:

- **numpy:** para operações matemáticas
- **pandas:** para manipulação de dados
- **sklearn.model_selection** e **sklearn.metrics:** para treinamento e avaliação de modelos
- **textblob:** para processamento de texto
- **sklearn.ensemble:** para utilizar o algoritmo de Gradient Boosting

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from textblob import TextBlob
from sklearn.ensemble import GradientBoostingRegressor

## 1. Carregando Conjuntos de Dados

Após a importação das bibliotecas, carregamos os conjuntos de dados necessários para o projeto. Estes são armazenados em DataFrames do Pandas para facilitar a manipulação posterior.

In [2]:
path = '../data/input/commonlit-evaluate-student-summaries/'
train_data = pd.read_csv(path + 'summaries_train.csv')
test_data = pd.read_csv(path + 'summaries_test.csv')
prompts_train = pd.read_csv(path + 'prompts_train.csv')
prompts_test = pd.read_csv(path + 'prompts_test.csv')
sample_submission = pd.read_csv(path + 'sample_submission.csv')

### 1.1. Visualização Inicial dos Dados

Em seguida, visualizamos as primeiras linhas de cada conjunto de dados para ter uma ideia inicial da estrutura e dos tipos de dados que estamos lidando.

In [3]:
datasets = {
    "Summaries Train": train_data,
    "Summaries Test": test_data,
    "Prompts Train": prompts_train,
    "Prompts Test": prompts_test,
}

for name, dataset in datasets.items():
    print(f"\n{name} - Primeiras linhas:")
    display(dataset.head())


Summaries Train - Primeiras linhas:


,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757



Summaries Test - Primeiras linhas:


,student_id,prompt_id,text
0,000000ffffff,abc123,Example text 1
1,111111eeeeee,def789,Example text 2
2,222222cccccc,abc123,Example text 3
3,333333dddddd,def789,Example text 4



Prompts Train - Primeiras linhas:


,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."



Prompts Test - Primeiras linhas:


,prompt_id,prompt_question,prompt_title,prompt_text
0,abc123,Summarize...,Example Title 1,Heading\nText...
1,def789,Summarize...,Example Title 2,Heading\nText...


### 1.2. Estatísticas Básicas

Por fim, geramos algumas estatísticas básicas para cada conjunto de dados. Isso pode nos dar insights sobre a distribuição dos dados, valores faltantes, entre outros aspectos importantes.

In [4]:
for name, dataset in datasets.items():
    print(f"\n{name} - Estatísticas básicas:")
    display(dataset.describe(include='all'))


Summaries Train - Estatísticas básicas:


,student_id,prompt_id,text,content,wording
count,7165,7165,7165,7165.000000,7165.000000
unique,7165,4,7165,NaN,NaN
top,000e8c3c7ddb,39c16e,The third wave was an experimentto see how peo...,NaN,NaN
freq,1,2057,1,NaN,NaN
mean,NaN,NaN,NaN,-0.014853,-0.063072
std,NaN,NaN,NaN,1.043569,1.036048
min,NaN,NaN,NaN,-1.729859,-1.962614
25%,NaN,NaN,NaN,-0.799545,-0.872720
50%,NaN,NaN,NaN,-0.093814,-0.081769
75%,NaN,NaN,NaN,0.499660,0.503833



Summaries Test - Estatísticas básicas:


,student_id,prompt_id,text
count,4,4,4
unique,4,2,4
top,000000ffffff,abc123,Example text 1
freq,1,2,1



Prompts Train - Estatísticas básicas:


,prompt_id,prompt_question,prompt_title,prompt_text
count,4,4,4,4
unique,4,4,4,4
top,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
freq,1,1,1,1



Prompts Test - Estatísticas básicas:


,prompt_id,prompt_question,prompt_title,prompt_text
count,2,2,2,2
unique,2,1,2,1
top,abc123,Summarize...,Example Title 1,Heading\nText...
freq,1,2,1,2


### 0.2. Importação de Bibliotecas Adicionais e Preparação de Funções

Agora, importamos bibliotecas adicionais para processamento de texto e definimos uma função para limpar e normalizar o texto.

- **ENGLISH_STOP_WORDS:** contém uma lista de palavras em inglês que são geralmente ignoradas no processamento de texto (stopwords).
- **PorterStemmer:** é um algoritmo de stemming para reduzir palavras à sua forma raiz.
- **re:** é a biblioteca de expressões regulares para manipulação de strings.

In [5]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem import PorterStemmer
import re

## 2. Função para Limpeza e Normalização de Texto

Definimos uma função chamada **`clean_and_normalize_text`** que executa várias etapas de limpeza e normalização em um texto dado.

In [6]:
def clean_and_normalize_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    text = text.lower()
    text = ' '.join(word for word in text.split() if word not in ENGLISH_STOP_WORDS)
    ps = PorterStemmer()
    text = ' '.join(ps.stem(word) for word in text.split())
    return text

### 2.1. Aplicando Limpeza e Normalização aos Datasets

Em seguida, aplicamos a **`função clean_and_normalize_text`** aos textos nos conjuntos de dados de treinamento e teste.

In [7]:
train_data['text_clean'] = train_data['text'].apply(clean_and_normalize_text)
test_data['text_clean'] = test_data['text'].apply(clean_and_normalize_text)

### 2.2. Mesclando Dados de Prompts

Depois de limpar os textos, mesclamos os dados de prompts nos conjuntos de dados de treinamento e teste usando o campo **`prompt_id`**.

In [8]:
train_data = train_data.merge(prompts_train, on='prompt_id', how='left')
test_data = test_data.merge(prompts_test, on='prompt_id', how='left')

### 2.3. Limpeza e Normalização de Prompts

Finalmente, aplicamos a limpeza e a normalização também aos textos dos prompts.

In [9]:
train_data['prompt_text_clean'] = train_data['prompt_text'].apply(clean_and_normalize_text)
test_data['prompt_text_clean'] = test_data['prompt_text'].apply(clean_and_normalize_text)

### 2.4. Visualização dos Textos Limpos

Para verificar o resultado da limpeza e normalização, podemos exibir as primeiras linhas do conjunto de treinamento, mostrando as versões originais e limpas dos textos e dos prompts.

In [10]:
train_data[['text', 'text_clean', 'prompt_text', 'prompt_text_clean']].head()

,text,text_clean,prompt_text,prompt_text_clean
0,The third wave was an experimentto see how peo...,wave experimentto peopl react new leader gover...,Background \r\nThe Third Wave experiment took ...,background wave experi took place cubberley hi...
1,They would rub it up with soda to make the sme...,rub soda make smell away wouldnt bad smell mea...,"With one member trimming beef in a cannery, an...",member trim beef canneri work sausag factori f...
2,"In Egypt, there were many occupations and soci...",egypt occup social class involv daytoday live ...,Egyptian society was structured like a pyramid...,egyptian societi structur like pyramid god ra ...
3,The highest class was Pharaohs these people we...,highest class pharaoh peopl godsthen nd highes...,Egyptian society was structured like a pyramid...,egyptian societi structur like pyramid god ra ...
4,The Third Wave developed rapidly because the ...,wave develop rapidli student genuinli believ b...,Background \r\nThe Third Wave experiment took ...,background wave experi took place cubberley hi...


## 3. Definindo Funções para Extração de Características Textuais

Agora, vamos definir algumas funções que serão úteis para extrair características (features) textuais que podem ser informativas para os modelos de aprendizado de máquina.

### 3.1. Função para Sobreposição de Palavras

Essa função calcula a sobreposição de palavras entre dois textos fornecidos. Ela retorna tanto o número de palavras em comum quanto a porcentagem de palavras do primeiro texto que estão presentes no segundo.

In [11]:
def word_overlap(text1, text2):
    words1 = set(str(text1).split())
    words2 = set(str(text2).split())
    overlap = words1.intersection(words2)
    if len(words1) == 0:
        return 0, 0.0
    return len(overlap), len(overlap) / len(words1)

### 3.2. Função para Diversidade Lexical

Essa função calcula a diversidade lexical de um texto, que é a razão entre o número de palavras únicas e o número total de palavras.

In [12]:
def lexical_diversity(text):
    words = str(text).split()
    unique_words = set(words)
    if len(words) == 0:
        return 0.0
    return len(unique_words) / len(words)

### 3.3. Função para Frequência de Palavras de Função

Esta função calcula a frequência de palavras de função (como "the", "and", etc.) em um texto.

In [13]:
function_words = {'the', 'and', 'to', 'of', 'a', 'in', 'that', 'it', 'is', 'was', 'for', 'on', 'with', 'as', 'by', 'at', 'an', 'be', 'this', 'which'}

def function_word_frequency(text):
    words = str(text).split()
    if len(words) == 0:
        return 0.0
    return sum(1 for word in words if word.lower() in function_words) / len(words)

### 3.4. Aplicando as Funções para Criar Novas Características

Depois de definir essas funções, aplicamos elas para criar novas colunas nos DataFrames de treinamento e teste.

In [14]:
train_data['summary_length'] = train_data['text'].apply(lambda x: len(str(x).split()))
train_data['word_overlap_count'], train_data['word_overlap_percent'] = zip(*train_data.apply(lambda row: word_overlap(row['text_clean'], row['prompt_text_clean']), axis=1))
train_data['lexical_diversity'] = train_data['text_clean'].apply(lexical_diversity)
train_data['function_word_frequency'] = train_data['text_clean'].apply(function_word_frequency)

test_data['summary_length'] = test_data['text'].apply(lambda x: len(str(x).split()))
test_data['word_overlap_count'], test_data['word_overlap_percent'] = zip(*test_data.apply(lambda row: word_overlap(row['text_clean'], row['prompt_text_clean']), axis=1))
test_data['lexical_diversity'] = test_data['text_clean'].apply(lexical_diversity)
test_data['function_word_frequency'] = test_data['text_clean'].apply(function_word_frequency)

### 3.5. Visualização das Novas Características

Para verificar se as novas características foram criadas corretamente, podemos exibir as primeiras linhas do conjunto de treinamento com essas novas colunas.

In [15]:
train_data[['summary_length', 'word_overlap_count', 'word_overlap_percent', 'lexical_diversity', 'function_word_frequency']].head()

,summary_length,word_overlap_count,word_overlap_percent,lexical_diversity,function_word_frequency
0,61,10,0.400000,0.806452,0.000000
1,52,12,0.857143,0.823529,0.000000
2,235,54,0.620690,0.813084,0.009346
3,25,7,0.583333,0.857143,0.000000
4,203,26,0.356164,0.793478,0.010870


### 0.3. Importação de Bibliotecas NLTK e Preparação de Recursos

Agora, vamos importar recursos adicionais da biblioteca NLTK (Natural Language Toolkit) para análise de texto. Vamos utilizar essa biblioteca para realizar a análise morfossintática (Part-of-Speech tagging, ou POS tagging) das palavras.

In [16]:
import nltk
from nltk.tokenize import word_tokenize

#### 0.3.1. Download de Recursos do NLTK

Antes de começar, é necessário baixar alguns recursos específicos do NLTK. Isso só precisa ser feito uma vez.

In [17]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to C:\Users\Eduardo
[nltk_data]     Froelich\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Eduardo Froelich\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## 4. Função para Calcular Razões de Tipos de Palavras

Definimos uma função chamada **`word_type_ratios`** que calcula a proporção de diferentes tipos de palavras (substantivos, verbos e adjetivos) em um dado texto.

In [18]:
def word_type_ratios(text):
    text = str(text)
    words = word_tokenize(text)
    pos_tags = nltk.pos_tag(words)
    
    noun_count = len([word for word, pos in pos_tags if pos in ['NN', 'NNP', 'NNPS', 'NNS']])
    verb_count = len([word for word, pos in pos_tags if pos in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']])
    adj_count = len([word for word, pos in pos_tags if pos in ['JJ', 'JJR', 'JJS']])
    
    total_count = len(words)
    if total_count == 0:
        return 0.0, 0.0, 0.0
    
    noun_ratio = noun_count / total_count
    verb_ratio = verb_count / total_count
    adj_ratio = adj_count / total_count
    
    return noun_ratio, verb_ratio, adj_ratio

### 4.1. Aplicando a Função para Criar Novas Características

Após definir a função, aplicamos ela para criar novas colunas nos DataFrames de treinamento e teste, representando as proporções de substantivos, verbos e adjetivos.

In [19]:
train_data['noun_ratio'], train_data['verb_ratio'], train_data['adj_ratio'] = zip(*train_data['text'].apply(word_type_ratios))
test_data['noun_ratio'], test_data['verb_ratio'], test_data['adj_ratio'] = zip(*test_data['text'].apply(word_type_ratios))

## 5. Sentiment Analysis com TextBlob

Agora, vamos realizar uma análise de sentimento para cada texto usando a biblioteca TextBlob. Isso nos fornecerá informações sobre a polaridade e a subjetividade do texto.

### 5.1. Função para Análise de Sentimento

Definimos uma função **`sentiment_analysis`** que utiliza o TextBlob para calcular a polaridade e a subjetividade de um texto dado.

In [20]:
def sentiment_analysis(text):
    text = str(text)
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    subjectivity = blob.sentiment.subjectivity
    return polarity, subjectivity

### 5.2. Aplicando a Função para Criar Novas Características

Utilizamos essa função para criar duas novas colunas nos DataFrames de treinamento e teste.

In [21]:
train_data['polarity'], train_data['subjectivity'] = zip(*train_data['text'].apply(sentiment_analysis))
test_data['polarity'], test_data['subjectivity'] = zip(*test_data['text'].apply(sentiment_analysis))